# [실습 프로젝트]

### **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

### 단계별 구현 가이드

- **Step 1**: 시스템 프롬프트 작성
   - 여행 플래너의 역할 명시
   - 구체적인 정보 포함 지시 (날짜, 장소, 예산 등)

- **Step 2**: 채팅 히스토리 관리
   - MessagesPlaceholder 사용
   - HumanMessage/AIMessage 변환

- **Step 3**: Gradio 인터페이스 설정
   - 제목과 설명 추가

**Step 4**: 테스트
   - "서울에서 2박 3일 여행 계획 짜줘" 등의 질문 시도

In [74]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

### 버전1

In [75]:
# 필수 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# 초기 메시지 정의
INITIAL_MESSAGE = (
    "✈️ 안녕하세요! 당신을 위한 맞춤 여행 계획을 만들어 드릴게요.\n"
    "여행하고 싶은 도시, 일정, 예산, 동행 인원을 알려주세요!"
)

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "당신은 맞춤형 여행 플래너 AI입니다.\n"
        "사용자의 입력을 바탕으로 다음을 반드시 포함해 한국어로 제안하세요:\n"
        "- 📅 일자별 일정\n"
        "- 💰 예상 예산(숙박/식비/교통/관광)\n"
        "- 🍽️ 현지 맛집 추천\n"
        "- 🚕 이동 방법\n"
        "- 🏨 숙소 유형 추천\n\n"
        "정보가 부족하면 먼저 필요한 항목을 질문하세요."
    ),
    MessagesPlaceholder("chat_history"),
    ("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.7,       # 적당한 창의성을 유지하면서 일관된 응답 생성
    top_p=0.9,             # 높은 확률의 토큰만 선택하여 응답의 품질 향상
    presence_penalty=0.3,  # 반복적인 응답을 줄이고 다양한 제안 생성(이미 많이 나온 단어의 재등장 억제)
    frequency_penalty=0.3, # 반복적인 응답을 줄이고 다양한 제안 생성(새로운 주제로의 전환 유도)
    streaming=True,
)

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):

    # 상태 메시지 출력
    yield "여행 계획을 생성 중입니다 😊"

    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    try:
        buffer = ""

        # 스트리밍 출력
        for chunk in chain.stream({
            "chat_history": history_messages,
            "user_input": message
        }):
            buffer += chunk
            yield buffer

    except Exception as e:
        yield f"⚠️ 오류가 발생했습니다: {str(e)}"



In [76]:
from pathlib import Path

# 현재 작업 디렉토리 기준
BASE_DIR = Path.cwd()          # 여기서 .ipynb 실행 중이면 notebook 위치 기준
DATA_DIR = BASE_DIR / "data"   # BASE_DIR/data
DATA_DIR.mkdir(exist_ok=True)  # 없으면 자동 생성

In [77]:
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont

# 🔤 한글 폰트 등록 (필수)
pdfmetrics.registerFont(UnicodeCIDFont("HYSMyeongJo-Medium"))

In [97]:
# 필수 라이브러리 임포트
import re
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, ListFlowable, ListItem
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# PDF 생성용 스타일
styles = getSampleStyleSheet()
styles["Normal"].fontName = "HYSMyeongJo-Medium"
styles["Heading1"].fontName = "HYSMyeongJo-Medium"

body_style = ParagraphStyle(
    "Body",
    parent=styles["Normal"],
    fontName="HYSMyeongJo-Medium",
    fontSize=11,
    leading=16,
)

title_style = ParagraphStyle(
    "Title",
    parent=styles["Heading1"],
    fontName="HYSMyeongJo-Medium",
    fontSize=18,
)

# 마크다운 → PDF 변환
def md_to_paragraph(text: str):
    flowables = []
    for line in text.split("\n"):
        line = line.strip()
        if not line:
            flowables.append(Spacer(1, 6))
            continue

        # 제목 처리
        if line.startswith("###"):
            content = line[3:].strip()
            flowables.append(Paragraph(f"<b>{content}</b>", title_style))
            flowables.append(Spacer(1, 8))
            continue

        # 목록 처리
        if line.startswith("- "):
            content = line[2:].strip()
            content = re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", content)
            flowables.append(Paragraph(f"• {content}", body_style))
            continue

        # 일반 문단 처리
        content = re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", line)
        flowables.append(Paragraph(content, body_style))

    return flowables

# PDF 생성 함수
def create_itinerary_pdf(ai_text: str, filename="travel_itinerary.pdf"):
    path = DATA_DIR / filename
    story = []

    story.append(Paragraph("✈️ 맞춤 여행 일정표", title_style))
    story.append(Spacer(1, 12))
    story.extend(md_to_paragraph(ai_text))

    doc = SimpleDocTemplate(
        str(path),
        pagesize=A4,
        rightMargin=20 * mm,
        leftMargin=20 * mm,
        topMargin=20 * mm,
        bottomMargin=20 * mm,
    )
    doc.build(story)
    return str(path.resolve())


In [98]:
# PDF 생성용 키워드
required_keywords = ["📅", "💰", "🍽️", "🚕", "🏨"]
def check_itinerary(ai_text: str):
    return any(kw in ai_text for kw in required_keywords)

In [99]:
# Gradio UI
with gr.Blocks() as demo:

    gr.Markdown("## ✈️ 맞춤형 여행 일정 계획 어시스턴트")

    chatbot = gr.Chatbot(
        value=[{"role": "assistant", "content": INITIAL_MESSAGE}],
        height=420,
    )

    chat = gr.ChatInterface(
        fn=answer_invoke,
        chatbot=chatbot,
    )

    status = gr.Markdown()

    with gr.Row():
        reset_btn = gr.Button("🔄 새 대화 시작")
        save_btn = gr.Button("📥 일정 PDF 저장")

    # ---------- Reset ----------
    def reset_chat():
        return [{"role": "assistant", "content": INITIAL_MESSAGE}]

    reset_btn.click(
        fn=reset_chat,
        outputs=chatbot,
    )

    # ---------- PDF ----------
    has_itinerary = gr.State(False)  # 일정 생성 여부 플래그

    def make_pdf(history, has_itinerary_flag):
        last_ai = None

        # Chatbot 포맷 대응 (tuple / dict)
        for msg in reversed(history):
            if isinstance(msg, dict) and msg.get("role") == "assistant":
                last_ai = msg.get("content")
                break
            elif isinstance(msg, (list, tuple)) and len(msg) == 2:
                last_ai = msg[1]
                break

        if not last_ai:
            return "⚠️ 저장할 AI 답변이 없습니다.", False

        # 문자열 변환
        if isinstance(last_ai, dict) and "text" in last_ai:
            last_ai_text = last_ai["text"]
        elif isinstance(last_ai, list):
            last_ai_text = "\n".join(str(x) for x in last_ai)
        else:
            last_ai_text = str(last_ai)

        # 일정 키워드 확인
        if not check_itinerary(last_ai_text):
            return "⚠️ 아직 여행 일정이 생성되지 않았습니다. 여행 정보를 입력해주세요.", False

        path = create_itinerary_pdf(last_ai_text)
        has_itinerary_flag = True
        return f"✅ PDF 저장 완료: `{path}`", True

    save_btn.click(
        fn=make_pdf,
        inputs=[chatbot, has_itinerary],
        outputs=[status, has_itinerary],
    )


demo.launch()

* Running on local URL:  http://127.0.0.1:7899
* To create a public link, set `share=True` in `launch()`.


In [4]:
# Gradio 인터페이스 종료
demo.close()

Closing server running on port: 7865
